In [39]:
import ee
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
# Define your area of interest (AOI)

# Define the start and end years
start_year = 2010
end_year = datetime.now().year

# Load Landsat 8 Surface Reflectance collection
prague = ee.Geometry.Point(14.4378, 50.0755).buffer(1000)
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(prague) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Function to calculate NDVI
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Function to extract July imagery for a given year
def get_july_imagery(year):
    start_date = ee.Date.fromYMD(year, 7, 1)
    end_date = ee.Date.fromYMD(year, 7, 31)
    
    return collection.filterDate(start_date, end_date) \
        .map(add_ndvi) \
        .select('NDVI') \
        .mean() \
        .set('year', year)

# Create a list of years
years = ee.List.sequence(start_year, end_year)

# Map over the years to get July NDVI for each year
july_ndvi_collection = ee.ImageCollection.fromImages(
    years.map(lambda year: get_july_imagery(year))
)




In [40]:
july_ndvi_collection

In [ ]:
def get_historical_nvdi(collection: ee.ImageCollection):
    collection.getInfo()
    images = []
    for i in range(len(july_ndvi_collection.getInfo()["features"])):
        image_list = july_ndvi_collection.toList()
          # Change this to your desired index
        image =image_list.get(i)
        
        numpy_array = geemap.ee_to_numpy(
            image,
            region=prague,
            scale=10 
        )
        images.append(numpy_array)  
    
    return numpy_array
        
images = get_historical_nvdi(july_ndvi_collection)


In [28]:
%matplotlib tk
plt.imshow(numpy_array, cmap='RdYlGn')

In [12]:
# Convert the image collection to an array
ndvi_array = july_ndvi_collection.toArray()

# Get the NDVI array as a numpy array
ndvi_np_array = geemap.ee_to_numpy(
    ndvi_array, 
    region=prague, 
    scale=30
)

# Print the shape of the resulting array
print(f"NDVI array shape: {ndvi_np_array.shape}")

# Save the array to a file
np.save('july_ndvi_array_2010_2024.npy', ndvi_np_array)
print("NDVI array saved to july_ndvi_array_2010_2024.npy")

Exception: An internal error has occurred (request: 6ff4ddcc-5d65-4acd-8ad7-5cf395adce8a)